# Greenhouse gas emissions to concentrations: vectorised

dimensions:
(ensembles, timesteps, species, boxes)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from scipy.interpolate import interp1d
import pickle
from climateforcing.utils import mkdir_p

from fair21.constants.gases import molwt, burden_per_emission, lifetime, gas_list
from fair21.defaults import n_gas_boxes
from fair21.defaults.gases import (
    partition_fraction,
    pre_industrial_concentration,
    natural_emissions_adjustment,
    iirf_0,
    iirf_cumulative,
    iirf_temperature,
    iirf_airborne,
    iirf_horizon
)
from fair21.gas_cycle import (
    calculate_g,
    calculate_alpha
)
from fair21.gas_cycle.forward import step_concentration

In [ ]:
# stick me in the constants!
SCENARIO_AXIS = 0
SPECIES_AXIS = 1
TIME_AXIS = 2
GAS_BOX_AXIS = 3

In [ ]:
def calculate_g(
    lifetime,
    partition_fraction,
    iirf_horizon=100,

):
    """Calculate the `g` components of the gas cycle.

    See Leach et al. (2021), eq. (5)

    Inputs
    ------
    lifetime : float
        atmospheric burden lifetime of the greenhouse gas (yr).
    partition_fraction : float, default=1 or `np.ndarray` of float
        proportion of emissions of gas that go into each atmospheric box.
        Should be 1 or sum to 1 if array.
    iirf_horizon : float, default=100
        time horizon (yr) for time integrated impulse response function.

    Returns
    -------
    g0 : float
    g1 : float
    """

    g1 = np.sum(partition_fraction * lifetime * (1 - (1 + iirf_horizon/lifetime) * np.exp(-iirf_horizon/lifetime)), axis=GAS_BOX_AXIS, keepdims=True)
    g0 = np.exp(-1 * np.sum(partition_fraction*lifetime*(1 - np.exp(-iirf_horizon/lifetime)), axis=GAS_BOX_AXIS, keepdims=True)/g1)

    return g0, g1

In [ ]:
def calculate_alpha(
    cumulative_emissions,
    airborne_emissions,
    temperature,
    iirf_0,
    iirf_cumulative,
    iirf_temperature,
    iirf_airborne,
    g0,
    g1,
    iirf_max = 99.95,
):
    """
    Calculate greenhouse-gas time constant scaling factor.

    Parameters
    ----------
    cumulative_emissions : float
        GtC cumulative emissions since pre-industrial.
    airborne_emissions : float
        GtC total emissions remaining in the atmosphere.
    temperature : float
        K temperature anomaly since pre-industrial.
    iirf_0 : float
        pre-industrial time-integrated airborne fraction.
    iirf_cumulative : float
        sensitivity of time-integrated airborne fraction with atmospheric
        carbon stock.
    iirf_temperature : float
        sensitivity of time-integrated airborne fraction with temperature
        anomaly.
    iirf_airborne : float
        sensitivity of time-integrated airborne fraction with airborne
        emissions.
    g0 : float
        parameter for alpha TODO: description
    g1 : float
        parameter for alpha TODO: description
    iirf_max : float
        maximum allowable value to time-integrated airborne fraction

    Returns
    -------
    alpha : float
        scaling factor for lifetimes
    """

    iirf = iirf_0 + iirf_cumulative * (cumulative_emissions-airborne_emissions) + iirf_temperature * temperature + iirf_airborne * airborne_emissions
    iirf = (iirf>iirf_max) * iirf_max + iirf * (iirf<iirf_max)
    alpha = g0 * np.exp(iirf / g1)

    return alpha

In [ ]:
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']

In [ ]:
# grab some emissions
emissions_array = np.ones((8, 43, 751, 1)) * np.nan
df = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
for iscen, scenario in enumerate(scenarios):
    for igas, gas in enumerate(gas_list):
        gas_rcmip_name = gas.replace("-", "")
        emissions_array[iscen, igas, ...] = df.loc[
            (df['Scenario']==scenario) & (df['Variable'].str.endswith("|"+gas_rcmip_name)) & (df['Region']=='World'), '1750':
        ].interpolate(axis=1).values.T
    
        # CO2 and N2O units need to behave
        if gas in ('CO2', 'N2O'):
            emissions_array[iscen, igas, :, :] = emissions_array[iscen, igas, :, :] / 1000

In [ ]:
emissions_array[7,21,:,:]

In [ ]:
# grab some concentrations
concentration_rcmip = {}
df = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
for scenario in scenarios:
    concentration_rcmip[scenario] = {}
    for gas in gas_list:
        gas_rcmip_name = gas.replace("-", "")
        concentration_rcmip[scenario][gas] = df.loc[
            (df['Scenario']==scenario) & (df['Variable'].str.endswith("|"+gas_rcmip_name)) & (df['Region']=='World'), '1750':
        ].interpolate(axis=1).values.squeeze()

In [ ]:
# grab indicative temperature projections
df = pd.read_csv('../data/rcmip-phase2/rcmip-phase2-fair162-ssp245-mean-temperature.csv')
ssp245_temperature_rfmip = df['temperature'].values

## Make plot similar to Nick's from FaIR 2.0.0-alpha

Some of the minor GHGs where natural emissions are not zero have misbehaviour after 2100. This is the fault of the scenario, not FaIR.

In [ ]:
n_gases = len(gas_list)
n_timesteps = 751
n_scenarios = len(scenarios)
n_gases

In [ ]:
# do we need to carry around an N-sized first dimension?
same_scenario = False
scenario_axis_size = (1-same_scenario) * (n_scenarios - 1) + 1
scenario_axis_size

In [ ]:
import numpy as np

def step_concentration(
    emissions,
    gas_boxes_old,
    airborne_emissions_old,
    burden_per_emission,
    lifetime,
    alpha_lifetime,
    partition_fraction,
    pre_industrial_concentration,
    timestep=1,
    natural_emissions_adjustment=0,
):
    """
    Calculates concentrations from emissions of any greenhouse gas.

    Parameters
    ----------
    emissions : ndarray
        emissions in timestep.
    gas_boxes_old : ndarray
        the greenhouse gas atmospheric burden in each lifetime box at the end of
        the previous timestep.
    airborne_emissions_old : ndarray
        The total airborne emissions at the beginning of the timestep. This is
        the concentrations above the pre-industrial control. It is also the sum
        of gas_boxes_old if this is an array.
    burden_per_emission : ndarray
        how much atmospheric concentrations grow (e.g. in ppm) per unit (e.g.
        GtCO2) emission.
    lifetime : ndarray
        atmospheric burden lifetime of greenhouse gas (yr). For multiple
        lifetimes gases, it is the lifetime of each box.
    alpha_lifetime : ndarray
        scaling factor for `lifetime`. Necessary where there is a state-
        dependent feedback.
    partition_fraction : ndarray
        the partition fraction of emissions into each gas box. If array, the
        entries should be individually non-negative and sum to one.
    pre_industrial_concentration : ndarray
        pre-industrial concentration of gas(es) in question.
    timestep : float, default=1
        emissions timestep in years.
    natural_emissions_adjustment : ndarray or float, default=0
        Amount to adjust emissions by for natural emissions given in the total
        in emissions files.

    Notes
    -----
    Emissions are given in time intervals and concentrations are also reported
    on the same time intervals: the airborne_emissions values are on time
    boundaries and these are averaged before being returned.
    
    Where array input is taken, the arrays always have the dimensions of
    (scenario, species, time, gas_box). Dimensionality can be 1, but we 
    retain the singleton dimension in order to preserve clarity of 
    calculation and speed.

    Returns
    -------
    concentration_out : ndarray
        greenhouse gas concentrations at the centre of the timestep.
    gas_boxes_new : ndarray
        the greenhouse gas atmospheric burden in each lifetime box at the end of
        the timestep.
    airborne_emissions_new : ndarray
        airborne emissions (concentrations above pre-industrial control level)
        at the end of the timestep.
    """

    decay_rate = timestep/(alpha_lifetime * lifetime)
    decay_factor = np.exp(-decay_rate)

    gas_boxes_new = (
        partition_fraction *
        (emissions-natural_emissions_adjustment) *
        1 / decay_rate *
        (1 - decay_factor) * timestep + gas_boxes_old * decay_factor
    )
    airborne_emissions_new = np.sum(gas_boxes_new, axis=GAS_BOX_AXIS, keepdims=True)
    concentration_out = (
        pre_industrial_concentration +
        burden_per_emission * (
            airborne_emissions_new + airborne_emissions_old
        ) / 2
    )

    return concentration_out, gas_boxes_new, airborne_emissions_new

In [ ]:
%%time

# initialise arrays. Using arrays makes things run quicker

# remember: scen, species, time, box
# where we don't care about keeping outputs we use a singleton dimension to maintain bookkeeping
# possible memory saver: differentiate between ensembles where we vary emissions and those where we vary climate parameters

# this scenario definition needs to be part of the initialisation, either from pyam, scmdata or fair emissions files
concentration_array = np.ones((n_scenarios, n_gases, n_timesteps, 1)) * np.nan
g0 = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
g1 = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
alpha_lifetime = np.ones((n_scenarios, n_gases, 1, 1))
airborne_emissions = np.zeros((n_scenarios, n_gases, 1, 1))
gas_boxes = np.zeros((n_scenarios, n_gases, 1, n_gas_boxes))
iirf_0_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
iirf_cumulative_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
iirf_temperature_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
iirf_airborne_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
burden_per_emission_array = np.ones((1, n_gases, 1, 1)) * np.nan
lifetime_array = np.ones((n_scenarios, n_gases, 1, n_gas_boxes)) * np.nan
pre_industrial_concentration_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
partition_fraction_array = np.zeros((n_scenarios, n_gases, 1, n_gas_boxes))
natural_emissions_adjustment_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan


cumulative_emissions_array = np.cumsum(emissions_array, axis=TIME_AXIS)
for igas, gas in enumerate(gas_list):
    lifetime_array[:, igas, :, :] = lifetime[gas]
    partition_fraction_array[:, igas, :, :] = partition_fraction[gas]
    iirf_0_array[:, igas, :, :] = iirf_0[gas]
    iirf_cumulative_array[:, igas, :, :] = iirf_cumulative[gas]
    iirf_temperature_array[:, igas, :, :] = iirf_temperature[gas]
    iirf_airborne_array[:, igas, :, :] = iirf_airborne[gas]
    burden_per_emission_array[:, igas, :, :] = burden_per_emission[gas]
    pre_industrial_concentration_array[:, igas, :, :] = pre_industrial_concentration[gas]
    partition_fraction_array[:, igas, :, :] = partition_fraction[gas]
    natural_emissions_adjustment_array[:, igas, :, :] = natural_emissions_adjustment[gas]

g0, g1 = calculate_g(lifetime_array, partition_fraction_array)

for i_timestep in range(n_timesteps):
    alpha_lifetime = calculate_alpha(
        cumulative_emissions_array[:, :, [i_timestep], :],
        airborne_emissions[:, :, [0], :],
        ssp245_temperature_rfmip[i_timestep],
        iirf_0_array,#[:, :, [i_timestep], :],
        iirf_cumulative_array,#[:, :, [i_timestep], :],
        iirf_temperature_array,#[:, :, [i_timestep], :],
        iirf_airborne_array,#[:, :, [i_timestep], :],
        g0,
        g1,
    )
    alpha_lifetime[np.isnan(alpha_lifetime)]=1  # CF4 seems to have an issue. Should we raise warning?
    concentration_array[:, :, [i_timestep], :], gas_boxes, airborne_emissions = step_concentration(
        emissions_array[:, :, [i_timestep], :], 
        gas_boxes,
        airborne_emissions, 
        burden_per_emission_array,
        lifetime_array,
        alpha_lifetime=alpha_lifetime,
        pre_industrial_concentration=pre_industrial_concentration_array,
        timestep=1,
        partition_fraction=partition_fraction_array,
        natural_emissions_adjustment=natural_emissions_adjustment_array,
    )

In [ ]:
for i in range(8):
    pl.plot(concentration_array[i,21,:,0], label=scenarios[i])
pl.legend();

In [ ]:
# put back together
concentration = {}

for iscen, scenario in enumerate(scenarios):
    concentration[scenario] = {}
    for igas, gas in enumerate(gas_list):
        concentration[scenario][gas] = concentration_array[iscen, igas, ...]

In [ ]:
fig, ax = pl.subplots(6, 8, figsize=(16,16))
for igas, gas in enumerate(gas_list):
    iy = igas % 8
    ix = igas // 8
    for iscen, scenario in enumerate(scenarios):
        ax[ix, iy].plot(np.arange(1750.5, 2501), concentration[scenario][gas], label='FaIR 2.1', color='k')
        ax[ix, iy].plot(np.arange(1750.5, 2501), concentration_rcmip[scenario][gas], label='History + MAGICC6', color='r')
    ax[ix, iy].set_title(gas)
fig.tight_layout()

In [ ]:
# would be awesome to now convert this to csv
# TODO: create scmdata or pyam dumper
mkdir_p("../data/output/")
with open("../data/output/rcmip-fair21-concentrations.pkl","wb") as fileout:
    pickle.dump(concentration, fileout)